In [1]:
import pandas as pd
from rdkit import Chem

In [2]:
def process_mol(mol):
    
    props = mol.GetPropsAsDict()
    try: cas = props['CAS Registry Numbers']
    except KeyError: cas = None
    
    return pd.Series({
            'SMILES' : Chem.MolToSmiles(mol, isomericSmiles=True),
            'Name' : props['ChEBI Name'],
            'ChEBI' : props['ChEBI ID'],
            'CAS' : cas,
        })

In [3]:
mols = pd.Series(Chem.SDMolSupplier('chebi_subset.sdf'))
mol_df = mols.apply(process_mol)

In [27]:
dragon_descriptors = pd.read_csv('chebi_dragon_output.tsv.gz', sep='\t', index_col=0)
dragon_descriptors.index = mol_df.SMILES
dragon_descriptors.drop('NAME', 1, inplace=True)
de_duplicated = dragon_descriptors[~dragon_descriptors.index.duplicated()]

de_duplicated.to_pickle('../ysi_utils/descriptors/dragon_chebi.p')

In [28]:
mol_df.to_pickle('../ysi_utils/data/chebi.p')